# Generate All Maximal Non-Branching Paths in a Graph

Reference : [Here](http://rosalind.info/problems/ba3m/)

#### Input  : The adjacency list of a graph whose nodes are integers.
#### Output : The collection of all maximal non-branching paths in the graph.

In [1]:
import pandas as pd
import numpy as np
import os

## Download the sample dataset from this [link](http://bioinformaticsalgorithms.com/data/extradatasets/assembly/maximal_nonbranching_paths.txt).

In [2]:
dataset = 'maximal_nonbranching_paths.txt'
with open(dataset) as file:
    raw_data = [line.rstrip() for line in file]
data = raw_data[1:raw_data.index('Output')]
out = raw_data[raw_data.index('Output')+1:]

## Pseudocode

    MaximalNonBranchingPaths(Graph)
        Paths ← empty list
        for each node v in Graph
            if v is not a 1-in-1-out node
                if out(v) > 0
                    for each outgoing edge (v, w) from v
                        NonBranchingPath ← the path consisting of the single edge (v, w)
                        while w is a 1-in-1-out node
                            extend NonBranchingPath by the outgoing edge (w, u) from w 
                            w ← u
                        add NonBranchingPath to the set Paths
        for each isolated cycle Cycle in Graph
            add Cycle to Paths
        return Paths 

In [3]:
# data = [
#     '1 -> 2',
#     '2 -> 3',
#     '3 -> 4,5',
#     '6 -> 7',
#     '7 -> 6'
# ]

*Convert_to_graph_edges* : this function converts the raw data into graph edges by storing each node and it's respective outgoing nodes in a dictionary.Here each node will be represented as the key and its outgoing nodes will be its value.



In [4]:
def convert_to_graph_edges(raw_data):
    graph = {}
    for line in raw_data:
        x = line.split(" -> ")
        a, b = x[0],x[1].split(',')
        subset = graph.get(x[0],[])
        graph[x[0]] = subset+b
        
    return graph

Load_adj_mat : This function creates an adjacency matrix by taking the edges of the graph as the parameter. 
    Firstly , the graph edges that were stored in the form of dictionary is stored in a set. 
    Then it is sorted.(ascending, lexographically). 
    Later a dataframe is created by using the node valuse as rows and columns
    The cells that are a part of the key index and come under value columns, are marked one. 
    This is how the adjacency matrix is created.

In [5]:
def load_adj_mat(graph_edges): # this method creates adjacent matrix
    st = set()
    for key in graph_edges.keys():
        st.add(key)
        for value in graph_edges[key]:
            st.add(value)
    
    st = list(np.sort([int(x) for x in list(st)]))
    st = [str(x) for x in st] # lexicographical order
    adj_mat = pd.DataFrame(np.zeros((len(st),len(st)),dtype=int),index=list(st),columns=list(st))
    
    for key in graph_edges.keys():
        for value in graph_edges[key]:
            adj_mat.loc[key,value] = 1
            
    #print(adj_mat)
    return adj_mat

do_DFS : Using the Adjacency matrix, we fisrt detect the edges,find the outgoing nodes and append it to a list. If there are no out going nodes, we return the list.
Everytime we come across a new outgong node, we check whether that node is present in the list or not. If it is not present in the list, We perform DFS for that node. This happens recursively.

i_dfs :This function is used to detect the isolated cycles. Intially the path will only consist of the starting node which is u. Since we are traversing through as isolated cycle, the degree of all the node will be 1-in-1-out. Primarily the status of all the nodes will be white. we change the status of the starting node to Grey and as we traverse the cylce ,we'll be changing the status of the visited nodes to grey and the visited node will become the new 'u' . Till the travel encounters a node with status as grey, it will go on. Once it encounters a grey node, the cycle(path) will be retuned.

In [6]:
def do_DFS(matrix,start,List):
    List.append(start)
    outgoing_nodes = matrix.index[matrix.loc[start].isin([1])].tolist() #edge detection

    if (outgoing_nodes == []):
        return List
    
    for outnode in outgoing_nodes:
        if outnode not in List:
            List = do_DFS(matrix,outnode,List) # Recursive Call
    return List

#For isolated cycle detection
def i_dfs(matrix,u,status):
    path = "" + u
    while True:
        status[u] = 'G'
        v = matrix.columns[matrix.loc[u].isin([1])].tolist()[0]
        
        path += " -> " + v

        if status[v] == 'G':
            break
        
        u = v
    return path

In [7]:
# def add_cycle(u,parent,ipaths):
#     start = u
#     path = f'{u} -> {parent[u]}'
#     u = parent[u]
#     while u!=start:
#         path = path + f' -> {parent[u]}'
#         u = parent[u]
#     ipaths.append(path)

MaximunNonBrancingPaths : This is the main function to find the maximum non-branching paths. Graph is taken as the parameter for this function. Firstly, the adjacency matrix of the graph is loaded and two lists called paths and icycle_paths ,a dictionary called status and a set are initialised. In the function, the keys i.e the nodes that are not one-in-one-out are found. If the outgoing nodes of the node(key) is greater than 0, then that node(key) is added to the set. All the outgoing nodes of the respective node is added to the list call paths(along with the node itself). While the out_going_node is a one-in-one-out node , we extend the NonBranchingPath and add that outnode to the set. 

Next we'll be checking for the isolated cycles. For this we find the nodes(keys) that are part of the matrix but not the set.These nodes are defined under the variable name icycle_nodes. All the icycle_nodes are initialised with status as white.
Now these nodes are traversed one by one.whenever a node with status white is encountered,i_dfs function is called and the respective node is appended to the icycle_paths list.


In [8]:
def MaximalNonBranchingPaths(graph):
    matrix = load_adj_mat(graph)
    Paths = []
    Set = set()
    
    #ICycle inits
    status = {}
    icycle_paths = []
    
    for key in graph.keys():
        if (not(matrix.loc[key].sum() == 1 and matrix[key].sum() == 1)):
            if (matrix.loc[key].sum() > 0):
                outgoing_nodes = matrix.index[matrix.loc[key].isin([1])].tolist()
                Set.add(key)
                for outnode in outgoing_nodes:
                    path = key + ""
                    while(matrix.loc[outnode].sum()==1 and matrix[outnode].sum()==1):
                        Set.add(outnode)
                        path = path + " -> " + outnode
                        outnode = matrix.index[matrix.loc[outnode].isin([1])].tolist()[0]

                    Set.add(outnode)
                    Paths.append(path + " -> " + outnode)

    icycle_nodes = [key for key in graph.keys() if key not in Set]
    for node in icycle_nodes:
        status[node] = 'W'
    
    for node in icycle_nodes:
        if status[node] == 'W':
            icycle_paths.append(i_dfs(matrix,node,status))
    
    Paths += icycle_paths
    return (Paths,matrix,icycle_paths)

In [9]:
graph = convert_to_graph_edges(data)
#print(graph)
#graph = {'AT': ['TT'], 'TT': ['TA'], 'TA': ['AC', 'AC', 'AC'], 'AC': ['CG', 'CC', 'CA'], 'CG': ['GG'], 'GG': ['GT'], 'GT': ['TA'], 'CC': ['CC', 'CT'], 'CT': ['TA'], 'CA': ['AA']}
Paths, Matrix, ic = MaximalNonBranchingPaths(graph)

#print(Paths)

# comment below code if you are using text dataset instead of file input.
with open('maximal_nonbranching_paths_output.txt','w') as f:
    for path in Paths:
        f.write(path + '\n')

#print('Raw Data    :',data)
#print('Graph Edges :',graph)
#print('\nPaths :')
print(*Paths,sep='\n')
# # print('Adjacent Matrix :')
# Matrix

0 -> 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13
13 -> 14 -> 15 -> 16 -> 17 -> 18 -> 19 -> 20 -> 21 -> 22 -> 23 -> 24 -> 25 -> 26 -> 27
13 -> 27
27 -> 28 -> 29 -> 30 -> 31 -> 32 -> 33 -> 34
34 -> 35 -> 36 -> 37 -> 38 -> 39 -> 40 -> 41 -> 42 -> 43 -> 44 -> 45 -> 46 -> 47 -> 48
34 -> 48
48 -> 49 -> 50 -> 51 -> 52
52 -> 53 -> 54 -> 55 -> 56 -> 57 -> 58 -> 59 -> 60 -> 61
52 -> 61
61 -> 62 -> 63 -> 64 -> 65 -> 66 -> 67 -> 68 -> 69 -> 70 -> 71 -> 72
72 -> 73 -> 74 -> 75 -> 76 -> 77 -> 78 -> 79 -> 80 -> 81 -> 82 -> 83 -> 84 -> 85
72 -> 85
85 -> 86 -> 87 -> 88 -> 89 -> 90
90 -> 91 -> 92 -> 93 -> 94 -> 95 -> 96 -> 97 -> 98 -> 99 -> 100 -> 101 -> 102 -> 103 -> 104 -> 105
90 -> 105
105 -> 106 -> 107 -> 108 -> 109 -> 110 -> 111 -> 112 -> 113
113 -> 114 -> 115 -> 116 -> 117 -> 118 -> 119 -> 120 -> 121 -> 122
113 -> 122
122 -> 123 -> 124 -> 125 -> 126 -> 127 -> 128 -> 129 -> 130 -> 131 -> 132 -> 133 -> 134
134 -> 135 -> 136 -> 137 -> 138 -> 139 -> 140 -> 141 -> 142 -> 143 -> 14